In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 26 12:06:53 2020.

@author: malchiodi
"""

from mulearn import FuzzyInductor
from mulearn.kernel import PrecomputedKernel
from mulearn.fuzzifier import *
from mulearn.optimization import GurobiSolver
import csv
import numpy as np
import statistics
from sklearn.model_selection import KFold, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from mulearn.distributions import *

def get_kernel_and_solver(gram):
    eigvals = np.linalg.eigvals(gram)
    assert(sum([abs(e.imag) for e in eigvals]) < 1e-4)
    abs_neg_eigvals = [-l.real for l in eigvals if l < 0]
    adjustment = max(abs_neg_eigvals) if abs_neg_eigvals else 0

    kernel = PrecomputedKernel(gram)
    solver = GurobiSolver(adjustment=adjustment) if adjustment else GurobiSolver()

    return kernel, solver

def get_dataset(filename):
    with open(filename) as data_file:
        data = np.array(list(csv.reader(data_file)))

    n = len(data) - 1
    n = 100

    # ## Extract data names, membership values and Gram matrix

    names = np.array(data[0])[1:n+1]
    mu = np.array([float(row[0]) for row in data[1:n+1]])
    gram = np.array([[float(k.replace('NA', '0')) for k in row[1:n+1]]
                     for row in data[1:n+1]])

    assert(len(names.shape) == 1)
    assert(len(mu.shape) == 1)
    assert(len(gram.shape) == 2)

    assert(names.shape[0] == gram.shape[0] == gram.shape[1] == mu.shape[0])

    X = np.array([[x] for x in np.arange(n)])

    return X, gram, mu


data_file_name = 'data/data-tettamanzi-complete.csv'
X, gram, mu = get_dataset(data_file_name)


out_cv = KFold()

k, solver = get_kernel_and_solver(gram)

fuzzifiers = [CrispFuzzifier(), QuantileConstantPiecewiseFuzzifier(), QuantileLinearPiecewiseFuzzifier(), LinearFuzzifier(),ExponentialFuzzifier()]
mean_test_scores = []
dev_test_scores = []
mean_train_scores = []
dev_train_scores = []

for fuzzifier in fuzzifiers: 
    test_scores = []
    train_scores = []
    i = 1
    
    fi = FuzzyInductor(k=k, solver=solver, fuzzifier= fuzzifier)

    inner_folds = 5
    rmse = make_scorer(mean_squared_error)
    
    gs = GridSearchCV(fi, {'c': np.logspace(-1, 1, 7)},
                        verbose=0, cv=inner_folds,
                        error_score= np.nan, scoring = rmse, n_jobs= 1,
                        pre_dispatch=10, refit = True)

    for train_idx, test_idx in out_cv.split(X):
        X_train = X[train_idx]
        X_test = X[test_idx]
        mu_train = mu[train_idx]
        mu_test = mu[test_idx]

        try:
            gs.fit(X_train, mu_train)
            print(f"fold {i}: best parameters: {gs.best_params_['c']}")
            #e = rs.estimator.set_params(**rs.best_params_)
            train_score = gs.score(X_train, mu_train)
            test_score = gs.score(X_test, mu_test)
            print(f'fold {i}: train score {train_score:.2f}, test score {test_score:.2f}')
            test_scores.append(test_score)
            train_scores.append(train_score)
            i += 1
        except ValueError as e:
            print(e)
            test_scores.append(np.nan)
            train_scores.append(np.nan)
            i += 1
            continue
        
    mean_test_scores.append(np.nanmean(test_scores))
    mean_train_scores.append(np.nanmean(train_scores))
    dev_test_scores.append(np.nanstd(test_scores))
    dev_train_scores.append(np.nanstd(train_scores))

fold 1: best parameters: 0.1
fold 1: train score 0.00, test score 0.00
fold 2: best parameters: 0.1
fold 2: train score 0.00, test score 0.04
fold 3: best parameters: 0.1
fold 3: train score 0.00, test score 0.00
fold 4: best parameters: 0.1
fold 4: train score 0.00, test score 0.00
fold 5: best parameters: 0.1
fold 5: train score 0.00, test score 0.00
fold 1: best parameters: 0.1
fold 1: train score 0.06, test score 0.13
fold 2: best parameters: 0.1
fold 2: train score 0.06, test score 0.12
fold 3: best parameters: 0.1
fold 3: train score 0.06, test score 0.13
fold 4: best parameters: 0.1
fold 4: train score 0.06, test score 0.14
fold 5: best parameters: 0.1
fold 5: train score 0.06, test score 0.11
fold 1: best parameters: 0.1
fold 1: train score 0.05, test score 0.13
fold 2: best parameters: 0.1
fold 2: train score 0.05, test score 0.13
fold 3: best parameters: 0.1
fold 3: train score 0.05, test score 0.13
fold 4: best parameters: 0.1
fold 4: train score 0.05, test score 0.13
fold 5

In [3]:
import pandas as pd

rmse_test = []
for i in range(len(mean_test_scores)):
    rmse_test.append(str(round(mean_test_scores[i],5)) + " +/- " + str(2*round(dev_test_scores[i],3)))

rmse_train = []
for i in range(len(mean_train_scores)):
    rmse_train.append(str(round(mean_train_scores[i],3)) + ' +/- ' + str(2*round(dev_train_scores[i],3)))
    
d = {'RMSE test' : rmse_test, 'RMSE train' : rmse_train}
df = pd.DataFrame(d, index = ['CrispFuzzifier', 'QuantileConstantPiecewiseFuzzifier', 'QuantileLinearPiecewiseFuzzifier','LinearFuzzifier','ExponentialFuzzifier'])
df.head()

,RMSE test,RMSE train
CrispFuzzifier,0.0086 +/- 0.03,0.001 +/- 0.0
QuantileConstantPiecewiseFuzzifier,0.1249 +/- 0.018,0.059 +/- 0.002
QuantileLinearPiecewiseFuzzifier,0.12684 +/- 0.01,0.053 +/- 0.002
LinearFuzzifier,0.00458 +/- 0.014,0.001 +/- 0.002
ExponentialFuzzifier,0.00434 +/- 0.014,0.001 +/- 0.0
